In [2]:
import os
os.chdir('/home/gabo/Documents/Facultad-local/Urdimbres-Sofisticadas')

In [3]:
from lectura import ldata
import networkx as nx
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd


import sys
sys.path.append('./Tp2/')
from agregar_esencialidad import agregar_esencialidad, agregar_esencialidad_dict

#%%
apms = ldata('Tp2/tc02Data/yeast_AP-MS.txt')

y2h = ldata('Tp2/tc02Data/yeast_Y2H.txt')

lit = ldata('Tp2/tc02Data/yeast_LIT.txt')

lit_r = ldata('Tp2/tc02Data/yeast_LIT_Reguly.txt')
lit_r = [fila[:2] for fila in lit_r[1:]]

g_apms = nx.Graph()
g_apms.add_edges_from(apms)

g_lit = nx.Graph()
g_lit.add_edges_from(lit)

g_lit_reg = nx.Graph()
g_lit_reg.add_edges_from(lit_r)

g_y2h = nx.Graph()
g_y2h.add_edges_from(y2h)

ess = ldata('Tp2/tc02Data/Essential_ORFs_paperHe.txt')
ess =  ess[2:-4]
ess = [fila[1] for fila in ess]
ess = np.unique(ess)

for g in [g_apms, g_lit, g_lit_reg, g_y2h]:
    agregar_esencialidad_dict(g, ess)

def k_medio(g):
    N = g.order()
    k_med = sum(k for (nodo, k) in g.degree) / N
    return k_med

In [10]:
################ Punto B:

######## Tabla 1 de Zotenko: propiedades de las bases de datos

def fila_tabla_1(g):
    g = max(nx.connected_component_subgraphs(g), key=len)
    N = g.order()
    L = g.size()
    K = k_medio(g)
    Ci = np.average(list(nx.clustering(g).values()))
    return N, L, K, Ci

grafos = [g_lit, g_lit_reg, g_apms, g_y2h]
Ns, Ls, Ks, Cis = [], [], [], []
for g in grafos:
    N, L, K, Ci = fila_tabla_1(g)
    Ns.append(N)
    Ls.append(L)
    Ks.append(K)
    Cis.append(Ci)

tabla1 = pd.DataFrame(data={'Número de nodos':Ns,
                            'Número de enlaces':Ls,
                            'Grado medio':Ks,
                            'Coef. de clustering medio':Cis}, 
                      index=['Lit', 'Lit_reg', 'APMS', 'Y2H'])
pd.set_option('precision', 2)
tabla1 

,Número de nodos,Número de enlaces,Grado medio,Coef. de clustering medio
Lit,1213,2621,4.32,0.33
Lit_reg,3224,11789,7.31,0.27
APMS,1004,8319,16.57,0.65
Y2H,1647,2682,3.26,0.06


In [15]:
tabla2 = np.load('Tp2/tc02Data/tabla2zotenko.npz')['data']
tabla2 = pd.DataFrame(data=tabla2, columns=['Lit', 'Lit_reg', 'APMS', 'Y2H'],
                     index=['Lit', 'Lit_reg', 'APMS', 'Y2H'])
tabla2

,Lit,Lit_reg,APMS,Y2H
Lit,1.00,0.98,0.37,0.07
Lit_reg,0.22,1.00,0.17,0.03
APMS,0.12,0.25,1.00,0.01
Y2H,0.07,0.15,0.04,1.00
